In [34]:
# Importando las librerías que vamos a utilizar
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import textacy
from textacy.datasets import Wikipedia
from collections import Counter, defaultdict
import warnings; warnings.simplefilter('ignore')
import spacy

# graficos incrustados
%matplotlib inline

# función auxiliar
def leer_texto(texto):
    """Funcion auxiliar para leer un archivo de texto"""
    with open(texto, 'r') as text:
        return text.read()

In [35]:
# Cargando el modelo en español de spacy
nlp = textacy.data.spacy.load('es_core_web_md')
#nlp = spacy.load('es_core_news_sm')


# Análisis del libro La transformación de las Razas en América
Proyecto Gutemberg http://www.gutenberg.org/ebooks/26947

In [36]:
# Limpiar preludio y licencia de Project Gutenberg

path_texto = '/home/drobins/diplodatos/datasets/pg26947.txt'
texto = ''
name = 'LAS RAZAS EN AMÉRICA'
start =  '*** START OF THIS PROJECT GUTENBERG EBOOK '+ name +' ***\n'
stop = '*** END OF THIS PROJECT GUTENBERG EBOOK '+ name +' ***\n'

with open(path_texto) as fp:
    for line in iter(fp.readline, start):
        pass
    for line in iter(fp.readline, stop):
        texto += line
texto_procesado = nlp(texto)

In [37]:
# Estadísticas del texto
ts = textacy.TextStats(texto_procesado)
ts.basic_counts

{'n_sents': 1199,
 'n_words': 45649,
 'n_chars': 224808,
 'n_syllables': 89630,
 'n_unique_words': 8694,
 'n_long_words': 13851,
 'n_monosyllable_words': 22733,
 'n_polysyllable_words': 13210}

In [38]:
doc = textacy.Doc(texto_procesado)
bot = doc.to_bag_of_words(normalize=True,as_strings=True,weighting='count')
sorted(bot.items(), key=lambda x: x[1], reverse=True)[:10]


[('y', 1878),
 ('a', 917),
 ('o', 275),
 ('vida', 132),
 ('mundo', 97),
 ('hombre', 94),
 ('Y', 93),
 ('humana', 89),
 ('espíritu', 86),
 ('Dios', 81)]

In [39]:
stop = list(textacy.data.spacy.es.STOP_WORDS)
stop.append('y')
stop.append('a')
stop.append('o')

palabras = list(textacy.extract.words(doc, filter_stops=True, filter_punct=True, filter_nums=False, include_pos=None, exclude_pos=None, min_freq=1))

contar_palabras = Counter()
for word in palabras:
    if word.string.strip().lower() not in stop:
       contar_palabras[word.string.strip().lower()] += 1

contar_palabras.most_common(10)    
    


[('vida', 134),
 ('mundo', 98),
 ('hombre', 96),
 ('dios', 89),
 ('humana', 89),
 ('espíritu', 88),
 ('hombres', 80),
 ('moral', 58),
 ('civilización', 57),
 ('siglo', 52)]

In [40]:
def encontrar_personajes(doc):
    """
    Devuelve una lista de los personajes de un `doc` con su cantidad de
    ocurrencias
    
    :param doc: NLP documento parseado por Spacy
    :return: Lista de Tuplas con la forma
        [('winston', 686), ("o'brien", 135), ('julia', 85),]
    """
    
    personajes = Counter()
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            personajes[ent.lemma_] += 1
            
    return personajes.most_common()

In [41]:
encontrar_personajes(texto_procesado)

[('', 29),
 ('álvarez', 14),
 ('dios', 14),
 ('emerson', 9),
 ('agustín álvarez', 7),
 ('sarmiento', 7),
 ('renan', 6),
 ('hubbard', 6),
 ('alberdi', 5),
 ('constantino', 5),
 ('iglesia', 5),
 ('jesús', 4),
 ('mahoma', 4),
 ('rosas', 4),
 ('robertson', 4),
 ('voltaire', 3),
 ('cristo', 3),
 ('fiske', 3),
 ('según', 2),
 ('jaurés', 2),
 ('taine', 2),
 ('grant allen', 2),
 ('lourdes', 2),
 ('a. france', 2),
 ('moisés', 2),
 ('san \n pablo', 2),
 ('satán', 2),
 ('polibio', 2),
 ('maeterlinck', 2),
 ('confucio', 2),
 ('epicteto', 2),
 ('marco aurelio', 2),
 ('napoleón', 2),
 ('aristóteles', 2),
 ('césar', 2),
 ('galileo', 2),
 ('vélez sársfield', 2),
 ('mendoza', 1),
 ('educación moral', 1),
 ('agustín álvarez_.', 1),
 ('miral', 1),
 ('benavente', 1),
 ('ameghino', 1),
 ('darwin', 1),
 ('esquiú', 1),
 ('compréndese', 1),
 ('alicia \n moreau', 1),
 ('eticista', 1),
 ('m. bergeret', 1),
 ('miguel de unamuno', 1),
 ('joaquín \n v. gonzález', 1),
 ('ingenieros', 1),
 ('máximo victoria', 1),
 (

## Análisis propio 
Queremos buscar las diferentes organizaciones sobre las cuales habla el libro y analizar el texto de la más representativa

In [42]:
# Etiquetas del texto
set(token.pos_ for token in texto_procesado)

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SPACE',
 'SYM',
 'VERB'}

### Encontrar las organizaciones sobre las cuales habla el libro

In [43]:
# Entidades nombradas de tipo ORG
[ent for ent in texto_procesado.ents if ent.label_ == 'ORG']
def encontrar_entidades(doc,strTipo):
   """
   Devuelve una lista de los entidades de un `doc` con su cantidad de
   ocurrencias
   
   :param doc: NLP documento parseado por Spacy
   :return: Lista de Tuplas con la forma
       [('winston', 686), ("o'brien", 135), ('julia', 85),]
   """
   
   entidades = Counter()
   for ent in doc.ents:
       if ent.label_ == strTipo:
           entidades[ent.lemma_] += 1
           
   return entidades.most_common()
print(encontrar_entidades(texto_procesado,'ORG')[:20])

[('iglesia', 26), ('', 14), ('europa', 3), ('syllabus', 2), ('produced by chuck greif', 1), ('mayo 638', 1), ('consejo supremo de guerra', 1), ('advertencia de la', 1), ('arturo e. de la mota', 1), ('de la', 1), ('el alfarero y', 1), ('la iglesia', 1), ('alemania del norte', 1), ('universidad de chuquisaca', 1), ('inquisición española', 1), ('_ trivium _', 1), ('iglesia cristiana', 1), ('grecia', 1), ('palestina', 1), ('inquisición', 1)]


### Leer el texto donde habla de la iglesia

In [25]:
# sentencias con las que aparece el Gran Hermano
[sent for sent in texto_procesado.sents if 'iglesia' in sent.string][-5:]


[Impidiendo o prohibiendo la cultura intelectual y la tolerancia, que es
 la cultura moral, las iglesias cristianas que llevaban en sí el cielo y
 el infierno, la civilización y la barbarie, suprimieron las
 posibilidades mentales para las partes superiores de sus propias
 doctrinas, y éstas quedaron incomprendidas, en letra muerta, mientras
 eran letra viva las partes inferiores durante los diez siglos de la era
 precientífica, en los que la civilización cristiana, con infierno y
 diablos, brujas, duendes, hechicheros y magos, íncubos, sucubos, silfos,
 gnomos, etc. con servidumbre, esclavitud y torturas, no se distinguía de
 la judía o la musulmana sino por su mayor ferocidad.
 , En la primera forma, "el santo terror del infierno" cubrió de iglesias,
 conventos y ermitas el Asia Menor, el Egipto y la Europa; en la segunda,
 originó las cruzadas y las órdenes de caballería religiosa, engendró la
 Inquisición y los Jesuitas; en fin, suscitó las guerras intercristianas,
 en las que los 

### Adjetivos para la iglesia

In [31]:
def obtener_adj_org(doc, organizacion):
    """
    Encontrar todos los adjetivos relacionados a un personaje en un `doc`
    
    :param doc: NLP documento parseado por Spacy
    :param personaje: un objeto String 
    :return: lista de adjetivos relacionados a un `personaje`
    """
    
    adjetivos = []
    for ent in doc.ents:
        if ent.lemma_ == organizacion:
            for token in ent.subtree:
                if token.pos_ == 'ADJ':
                    adjetivos.append(token.lemma_)
    
    for ent in doc.ents:
        if ent.lemma_ == organizacion:
            if ent.root.dep_ == 'nsubj':
                for child in ent.root.head.children:
                    if child.dep_ == 'acomp':
                        adjetivos.append(child.lemma_)
    
    return adjetivos

In [33]:
obtener_adj_org(texto_procesado, 'iglesia')


['omnipotente', 'divina', 'sagradas', 'humana', 'libre']